# Python's built-in types

## Strings and bytes

В Py3 только один тип для строк - str. Строки не изменяемы и могут использоваться только для хранения Unicode текста.

Альтернатива строкам - bytes и их изменяемый вариант - bytearray. Любой элемент bytes может принимать значения целочисленные значения от 0 до 255 включительно

In [2]:
bytes([102, 111, 111])

b'foo'

In [6]:
print(list(b'foo bar'))
print(tuple(b'spam'))

[102, 111, 111, 32, 98, 97, 114]
(115, 112, 97, 109)


Префикс **b** или **B** означает то, что данные являются bytes, заданные ASCII символами. Если использовать не ASCII  - будет исключение.

In [5]:
b'Привет'

SyntaxError: bytes can only contain ASCII literal characters. (<ipython-input-5-72fd33a3f6ef>, line 1)

In [7]:
type(b'foo'), type('boo')

(bytes, str)

In [9]:
B'foo' == b'foo'

True

In [17]:
s = 'Привет'
print(s.encode())  # конвертируем строку в bytes
print(tuple(s.encode()))  # по умолчанию для кодирования используется кодировка utf-8

b'\xd0\x9f\xd1\x80\xd0\xb8\xd0\xb2\xd0\xb5\xd1\x82'
(208, 159, 209, 128, 208, 184, 208, 178, 208, 181, 209, 130)


In [18]:
print([bin(i) for i in s.encode()])

['0b11010000', '0b10011111', '0b11010001', '0b10000000', '0b11010000', '0b10111000', '0b11010000', '0b10110010', '0b11010000', '0b10110101', '0b11010001', '0b10000010']


In [19]:
bytes('Привет', 'utf-8')  # преобразование строки в байты

b'\xd0\x9f\xd1\x80\xd0\xb8\xd0\xb2\xd0\xb5\xd1\x82'

In [25]:
b = bytes((208, 159, 209, 128, 208, 184, 208, 178, 208, 181, 209, 130))
print(b.decode())  # utf-8 по умолчанию
print(str(b, 'utf-8'))

Привет
Привет


А так работать с bytearray

In [31]:
ba = bytearray(b)
print(ba)
old_id = id(ba)
ba[2] = 233  # bytearray можно изменять
print(ba)
id(ba) == old_id  # id остается тем же

bytearray(b'\xd0\x9f\xd1\x80\xd0\xb8\xd0\xb2\xd0\xb5\xd1\x82')
bytearray(b'\xd0\x9f\xe9\x80\xd0\xb8\xd0\xb2\xd0\xb5\xd1\x82')


True

# Advanced syntax

## Iterators и generators

Итератор - это объект, который реализует протокол итератора.

**Протокол итератора**

Это просто два магических метода:

* __next__ : возвращает следующий элемент
* __iter__ : возвращает сам итератор

In [33]:
class CountDown:
    def __init__(self, step):
        self.step = step
    def __next__(self):
        if self.step <= 0:
            raise StopIteration
        self.step -= 1
        return self.step
    def __iter__(self):
        return self

In [34]:
[elem for elem in CountDown(10)]

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

**yield** выражение

In [55]:
def count_down(start):
    position = start
    while position > 0:
        position -= 1
        yield position

In [56]:
type(count_down)

function

In [57]:
g = count_down(10)

In [58]:
type(g)

generator

In [59]:
i = iter(g)

In [60]:
type(i)

generator

In [61]:
i is g

True

In [62]:
next(g)

9

In [63]:
next(g)

8

In [65]:
[next(item) for item in g]

[]

# Менеджеры контекста

Менеджеры контекста должны поддерживать соответствующий протокол, т.е. иметь в наличии два 
магических метода:

* __enter__(self) - вход в контекст
* __exit__(self) - срабатывает на выходе из контекста

Обычный порядок работы:
1. Выполняется метод __enter__
2. Выполняется тело контекста
3. Выполняется __exit__

По сути менеджер контекста, это альтернатива такой конструкции:

```
hosts = open('/etc/hosts/')
try:
    for line in hosts:
        if line.startswith('#'):
             continue
        print(line.strip())
finally:
    hosts.close()
```

In [69]:
with open('README.md') as readme:
    for line in readme:
        if line.startswith('#'):
            continue
        print(line.strip())

In [83]:
class MyContext:
    def __init__(self, param):
        self.param = param
        print('Context created')
    def __enter__(self):
        print('entered context for %s' % self.param)
    def __exit__(self, exc_type, exc_value, traceback):
        print('leaving context')
        if exc_type is None:
            print('No errors')
        elif exc_type is ValueError:
            print('Value error. Shit happens. Skip it.')
            return True
        else:
            print('With error %s' % exc_value)

In [84]:
with MyContext('testtt') as context:
    print('context body')

Context created
entered context for testtt
context body
leaving context
No errors


In [85]:
with MyContext('error'):
    raise ValueError('raised EXCEPTION') 

Context created
entered context for error
leaving context
Value error. Shit happens. Skip it.


In [86]:
with MyContext('error'):
    raise KeyError('raised EXCEPTION') 

Context created
entered context for error
leaving context
With error 'raised EXCEPTION'


KeyError: 'raised EXCEPTION'